In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

1. Build a unsupervised algorithm to cluster customer data (recommended: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) . Write a short text of what is the key business takeaway of the recommendation.

In [ ]:
#importing required libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#loadind dataset
data= pd.read_csv('../input/customer-personality-analysis/marketing_campaign.csv', delimiter='\t')
dataset= data
dataset

Data Pre-processing

In [ ]:
#checking datatypes
dataset.dtypes

In [ ]:
#filling missing missing values
dataset.fillna(dataset.mean(), inplace=True)
dataset

In [ ]:
#finding unique values in the data
dataset.nunique()

In [ ]:
#finding total number of kids, total number of customers that accepted the campaign, total amount spent and total purchase
dataset['TotalKids']= dataset['Kidhome']+dataset['Teenhome']
dataset['AcceptedCmp']= dataset['AcceptedCmp1']+ dataset['AcceptedCmp2']+ dataset['AcceptedCmp3']+ dataset['AcceptedCmp4']+ dataset['AcceptedCmp5']
dataset['AmountSpent']= dataset['MntWines']+ dataset['MntFruits']+ dataset['MntMeatProducts']+ dataset['MntFishProducts']+ dataset['MntSweetProducts']+ dataset['MntGoldProds']
dataset['TotalPurchase']= dataset['NumDealsPurchases']+ dataset['NumWebPurchases']+ dataset['NumCatalogPurchases']+ dataset['NumStorePurchases']

#converting education and marital status to required format
dataset['Education'] = dataset['Education'].replace(['PhD','2n Cycle','Graduation', 'Master'],'PG')  
dataset['Education'] = dataset['Education'].replace(['Basic'], 'UG')
dataset['Marital_Status'] = dataset['Marital_Status'].replace(['Married', 'Together'],'Not_single')
dataset['Marital_Status'] = dataset['Marital_Status'].replace(['Divorced', 'Widow', 'Alone', 'YOLO', 'Absurd'],'Single')
dataset

In [ ]:
#removing unnecessary columns
dataset.drop(['ID','Year_Birth','Kidhome','Teenhome','Dt_Customer', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts',
              'MntSweetProducts', 'MntGoldProds', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2',
             'Z_CostContact', 'Z_Revenue','NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases'], axis=1, inplace = True)
dataset

In [ ]:
#finding datatypes
dataset.dtypes

In [ ]:
#encoding the categorical columns
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

cat_var= []
for i in dataset.columns:
    if (dataset[i].dtypes == "object"):
        cat_var.append(i)
print(cat_var)

In [ ]:
for i in cat_var:
    dataset[i]= dataset[[i]].apply(le.fit_transform)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaledval = scaler.fit_transform(dataset.values)
scaled_data = pd.DataFrame(scaledval, index=dataset.index, columns=dataset.columns)
scaled_data

K-Means Clustering

In [ ]:
#finding number of clusters
from sklearn.cluster import KMeans
wcss= []
for i in range(1, 11):
    kmeans= KMeans(n_clusters= i, init= 'k-means++', random_state= 30)
    kmeans.fit(scaled_data)
    wcss.append(kmeans.inertia_)
plt.figure(figsize=(15, 8))
plt.plot(range(1, 11), wcss)
plt.title('The Elbow Method to find number of clusters')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
kmeans= KMeans(n_clusters= 3,random_state= 30)

In [ ]:
kmeans.fit(scaled_data)
KMeans(n_clusters= 3, random_state= 30)

In [ ]:
pred= kmeans.predict(scaled_data)
dataset['Cluster']= pred

In [ ]:
dataset

In [ ]:
dataset.describe()

In [ ]:
#plotting based on income and purchase
plt.figure(figsize=(10, 8))
plt.ylim([0, 200000])
pl = sns.scatterplot(data= dataset,x= dataset["AmountSpent"], y= dataset["Income"], hue= dataset["Cluster"], palette= 'plasma')
pl.set_title("Customer clusters Based On Income And Purchase")
plt.legend()
plt.show()

In [ ]:
dataset[dataset['Cluster']==0].describe()

**This cluster contains mostly of customers with Post Graduate level of education, where some are married and some are not. Here, Income is high and so is the total amount spent. This cluster purchased the maximum number of items. Maximum customers in this cluster accepted the campaign.**

In [ ]:
dataset[dataset['Cluster']==1].describe()

**This cluster contains of customers with Post Graduate level of education as well as few Under Graduate, where no one is married. Here, Income is moderate and so is the total amount spent. This cluster purchased the moderate number of items. Very few customers in this cluster accepted the campaign.**

In [ ]:
dataset[dataset['Cluster']==2].describe()

**This cluster contains of customers with Post Graduate level of education as well as Under Graduate, where most customers are married. Here, Income is the lowest and so is the total amount spent. This cluster purchased the lowest number of items. Extremely few customers in this cluster accepted the campaign.**

**The key business takeaway here would be that campaigning to highly educated customers who are married and have a high income would result in maximum sale of products.**

2. Build a classification model (binomial or multinomial - recommended: xgboost python implementation) on the promotion done by the company. Run SHAP analysis on the model results, and write a short text of what would be your recommendation to business for the next round of campaigns.

**Data Pre-processing**

In [ ]:
dataset= pd.read_csv('../input/customer-personality-analysis/marketing_campaign.csv', delimiter='\t')
dataset

In [ ]:
dataset.fillna(dataset.mean(), inplace=True)
dataset

In [ ]:
dataset['TotalKids']= dataset['Kidhome']+dataset['Teenhome']
dataset['AcceptedCmp']= dataset['AcceptedCmp1']+ dataset['AcceptedCmp2']+ dataset['AcceptedCmp3']+ dataset['AcceptedCmp4']+ dataset['AcceptedCmp5']
dataset['AmountSpent']= dataset['MntWines']+ dataset['MntFruits']+ dataset['MntMeatProducts']+ dataset['MntFishProducts']+ dataset['MntSweetProducts']+ dataset['MntGoldProds']
dataset['TotalPurchase']= dataset['NumDealsPurchases']+ dataset['NumWebPurchases']+ dataset['NumCatalogPurchases']+ dataset['NumStorePurchases']
dataset['Education'] = dataset['Education'].replace(['PhD','2n Cycle','Graduation', 'Master'],'PG')  
dataset['Education'] = dataset['Education'].replace(['Basic'], 'UG')
dataset['Marital_Status'] = dataset['Marital_Status'].replace(['Married', 'Together'],'Not_single')
dataset['Marital_Status'] = dataset['Marital_Status'].replace(['Divorced', 'Widow', 'Alone', 'YOLO', 'Absurd'],'Single')
dataset

In [ ]:
dataset.drop(['ID','Year_Birth','Kidhome','Teenhome','Dt_Customer', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts',
              'MntSweetProducts', 'MntGoldProds', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2',
             'Z_CostContact', 'Z_Revenue','NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases'], axis=1, inplace = True)
dataset

# ****# XGBoost Classifier****

In [ ]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -3].values
print(X)
print(y)

In [ ]:
#Encoding the categorical columns
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,0]=le.fit_transform(X[:,0])
print(X)

In [ ]:
#Encoding the categorical columns
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,1]=le.fit_transform(X[:,1])
print(X)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ct= ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
X= np.array(ct.fit_transform(X))
print(X)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

**The XGBoost classifier shows a 100% accuracy of the model.**

# **SHAP Analysis on the model**

In [ ]:
import shap
shap_values = shap.Explainer(classifier).shap_values(X_train)
shap.summary_plot(shap_values, X_train, plot_type="bar")